In [ ]:
#import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving kaggle.json to kaggle (1).json
User uploaded file "kaggle (1).json" with length 67 bytes


In [ ]:
# Crear un directorio y mover el archivo `kaggle.json` allí.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Cambiar los permisos del archivo.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d  phucthaiv02/butterfly-image-classification

Dataset URL: https://www.kaggle.com/datasets/phucthaiv02/butterfly-image-classification
License(s): CC0-1.0
butterfly-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip butterfly-image-classification.zip

Archive:  butterfly-image-classification.zip
replace Testing_set.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
#Importacion de librerias

import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch import nn
from torchvision import models, transforms
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import psutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torchvision.transforms import Lambda
from tqdm import tqdm

from datetime import datetime


In [ ]:
# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)



Using device: cuda


In [ ]:
# Configuraciones específicas del dataset


train_img_dir = './train'
train_csv_file = 'Training_set.csv'
train_df = pd.read_csv(train_csv_file)


train_df.head(3)



,filename,label
0,Image_1.jpg,SOUTHERN DOGFACE
1,Image_2.jpg,ADONIS
2,Image_3.jpg,BROWN SIPROETA


In [ ]:
label_list=list(train_df['label'].unique())
print(label_list)
print(len(label_list))
num_classes= len(label_list)
print(f"Número de clases: {num_classes}")


['SOUTHERN DOGFACE', 'ADONIS', 'BROWN SIPROETA', 'MONARCH', 'GREEN CELLED CATTLEHEART', 'CAIRNS BIRDWING', 'EASTERN DAPPLE WHITE', 'RED POSTMAN', 'MANGROVE SKIPPER', 'BLACK HAIRSTREAK', 'CABBAGE WHITE', 'RED ADMIRAL', 'PAINTED LADY', 'PAPER KITE', 'SOOTYWING', 'PINE WHITE', 'PEACOCK', 'CHECQUERED SKIPPER', 'JULIA', 'COMMON WOOD-NYMPH', 'BLUE MORPHO', 'CLOUDED SULPHUR', 'STRAITED QUEEN', 'ORANGE OAKLEAF', 'PURPLISH COPPER', 'ATALA', 'IPHICLUS SISTER', 'DANAID EGGFLY', 'LARGE MARBLE', 'PIPEVINE SWALLOW', 'BLUE SPOTTED CROW', 'RED CRACKER', 'QUESTION MARK', 'CRIMSON PATCH', 'BANDED PEACOCK', 'SCARCE SWALLOW', 'COPPER TAIL', 'GREAT JAY', 'INDRA SWALLOW', 'VICEROY', 'MALACHITE', 'APPOLLO', 'TWO BARRED FLASHER', 'MOURNING CLOAK', 'TROPICAL LEAFWING', 'POPINJAY', 'ORANGE TIP', 'GOLD BANDED', 'BECKERS WHITE', 'RED SPOTTED PURPLE', 'MILBERTS TORTOISESHELL', 'SILVER SPOT SKIPPER', 'AMERICAN SNOOT', 'AN 88', 'ULYSES', 'COMMON BANDED AWL', 'CRECENT', 'METALMARK', 'SLEEPY ORANGE', 'PURPLE HAIRSTREA

In [ ]:
import os
import cv2
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image

# Asegúrate de que label_list contenga todas las categorías únicas del DataFrame antes de este bucle
label_list = sorted(train_df['label'].unique())  # Ajusta 'category' al nombre de tu columna

prepared_images = []
labels = []  # Lista para almacenar los índices de las etiquetas

for image in tqdm(os.listdir(train_img_dir), desc="Preparing Images"):
    # Filtro para encontrar la categoría de la imagen actual
    filtre = train_df['filename'] == image
    if not filtre.any():
        continue  # Si no hay coincidencia, salta esta imagen
    kategori = train_df[filtre].values[0][1]  # Asegúrate de que este índice es correcto

    # Buscar el índice de la categoría en label_list
    try:
        kat = label_list.index(kategori)
        labels.append(kat)
    except ValueError:
        print(f"Category {kategori} not found in label_list.")
        continue  # O puedes asignar una categoría por defecto

    # Leer, redimensionar y normalizar la imagen
    image_path = os.path.join(train_img_dir, image)
    resim_oku = cv2.imread(image_path)
    if resim_oku is None:
        continue  # Si por alguna razón la imagen no se puede leer, salta
    resim180 = cv2.resize(resim_oku, (224, 224))
    resim = resim180 / 255.0  # Normalización
    flattened_image = resim.flatten()  # Aplanar la imagen
    prepared_images.append(flattened_image)

# Convertir listas a arrays de numpy para procesamiento posterior
prepared_images = np.array(prepared_images)
labels = np.array(labels)


Preparing Images:   0%|          | 0/6499 [00:00<?, ?it/s]

In [ ]:
print(len(prepared_images))
print(len(labels))

6499
6499


In [ ]:
x=np.array(prepared_images)
y=np.array(labels)

x.shape

(6499, 150528)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.70,random_state=35)

x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,test_size=0.5,random_state=35)



In [ ]:
 x_train.shape


(4549, 150528)

In [ ]:
print(f"Máximo en etiquetas de entrenamiento: {y_train.max()}, Mínimo: {y_train.min()}")

Máximo en etiquetas de entrenamiento: 74, Mínimo: 0


Mis datos estan en array de NumPy que contiene 4549 ejemplos, donde cada ejemplo es un vector aplanado de 150528 elementos. Este vector aplanado representa imágenes que han sido transformadas desde su formato original de 2D (o 3D con canales) a un vector 1D.

In [ ]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape

((4549, 150528), (975, 150528), (975, 150528), (4549,), (975,), (975,))

In [ ]:
x_train = x_train.reshape(-1, 3, 224, 224)  # Reformato para el conjunto de entrenamiento
x_val = x_val.reshape(-1, 3, 224, 224)      # Reformato para el conjunto de validación
x_test = x_test.reshape(-1, 3, 224, 224)    # Reformato para el conjunto de prueba


In [ ]:
print(x_train.shape)

(4549, 150528)


In [ ]:
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_test.shape, y_val.shape

((4549, 150528), (975, 150528), (975, 150528), (4549,), (975,), (975,))

In [ ]:
# Supongamos que ya tienes x_train, x_val, y x_test definidos como tensores aplanados
x_train = x_train.reshape(-1, 3, 224, 224)  # Reformato para el conjunto de entrenamiento
x_val = x_val.reshape(-1, 3, 224, 224)      # Reformato para el conjunto de validación
x_test = x_test.reshape(-1, 3, 224, 224)    # Reformato para el conjunto de prueba





In [ ]:
# Crear TensorDatasets
# Asegurarte de que los datos están en el tipo y dispositivo correcto
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_val = torch.tensor(x_val, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)


import torch

# Asumiendo que y_train, y_val, y_test son numpy arrays y device está definido
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)


<ipython-input-30-73f4565c639a>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
<ipython-input-30-73f4565c639a>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val = torch.tensor(x_val, dtype=torch.float32).to(device)
<ipython-input-30-73f4565c639a>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test = torch.tensor(x_test, dtype=torch.float32).to(device)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset


# Crear TensorDatasets
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Asegurarse de que los DataLoaders están funcionando como se espera
for images, labels in train_loader:
    print(images.shape, labels.shape)  # Deberías ver (32, 3, 224, 224) y (32,) si el batch_size es 32
    break


torch.Size([32, 3, 224, 224]) torch.Size([32])


In [ ]:
 for images, labels in train_loader:
    print(images.shape)  # Debería mostrar algo como torch.Size([32, 3, 224, 224])
    break


torch.Size([32, 150528])


In [ ]:
print(x_train.shape)  # Debería mostrar (4549, 3, 224, 224)


(4549, 3, 224, 224)


### CNN

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Capa convolucional
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Capa de pooling
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  # Aplanar las características
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN(num_classes)  # Ajusta num_classes según tus necesidades
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 10  # Define el número de épocas

for epoch in range(num_epochs):
    model.train()  # Poner el modelo en modo de entrenamiento
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluar el modelo después de cada época
    model.eval()  # Poner el modelo en modo de evaluación
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, '
          f'Accuracy: {100 * correct / total:.2f}%')


Epoch [1/10], Loss: 4.1541, Accuracy: 4.41%
Epoch [2/10], Loss: 4.1780, Accuracy: 6.26%
Epoch [3/10], Loss: 3.7602, Accuracy: 10.97%
Epoch [4/10], Loss: 3.0674, Accuracy: 10.05%
Epoch [5/10], Loss: 1.7980, Accuracy: 13.64%
Epoch [6/10], Loss: 1.4752, Accuracy: 15.18%
Epoch [7/10], Loss: 2.3865, Accuracy: 17.23%
Epoch [8/10], Loss: 2.3791, Accuracy: 16.31%
Epoch [9/10], Loss: 1.9625, Accuracy: 16.41%
Epoch [10/10], Loss: 0.7238, Accuracy: 17.44%


In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


Test Accuracy: 18.56%
